In [ ]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [ ]:
df = pd.read_csv('merged.csv')

In [ ]:
df.head(5)

In [ ]:
#df2 = df[(df['lat'].isnull()==True) & (df['address'].isnull()==False)]

In [ ]:
#(df['lat'].isnull()==True) &
#key = 'xxx'
key = keys.google
counter = 0
# loop through all records with an address, but no lat/long
for i, row in df[(df['lat'].isnull() == True) &
                 (df['address'].isnull() == False)
                 #& (df['id']=='0061a00000GcQdL') #for test
                 ].iterrows():
    # get the lat and long from google maps api
    address = row.address.replace(' ', '+')
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'=&key='+key
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        # print(data)
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        # add it to the dataframe
        df.loc[df['id'] == row.id, 'lat'] = lat
        df.loc[df['id'] == row.id, 'lng'] = lng
        # print(address,lat,lng)
    except:
        print('request failure for row', i, data)
    counter += 1
    if counter % 10 == 0:
        print(counter, address, lat, lng)
        # break

In [ ]:
# df.head(3000)

In [ ]:
# number of uncompleted records
df[(df['lat'].isnull() == True) & (df['address'].isnull() == False)].shape

In [ ]:
# export to csv
df.to_csv('merged.csv', index_label=False)